Upload files to colab

Translation function

In [0]:
#!/usr/bin/env python
# coding: utf-8

# Translate sentences to English and save mapping dicitionary as json.

# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

# Parameters:
# *   sentences =
# *   dictpath =  json dictionary file mapping original sentence with translation
# 
# 

# In[ ]:


from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'pt-br')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        try:
            translations[item]
        except KeyError:
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Prepare input

In [29]:
from pprint import pprint
import pandas as pd
import os

sentences = list()
label_idx = [
             'sentence1',
             'sentence2',
            ]
length = 0

for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        #print(label)
        #print('\n'*2)

        sentences.extend(list(label))
        length = length + label.size


assert length == len(sentences)
sentences = set(sentences)

train.tsv
test.tsv
dev.tsv


In [0]:
#Ver as sentenças semelhantes

# erro 'Rosinha Matheus is an Evangelical Christian.'
#'Rosinha Matheus é um cristã evangélica.'
#sentences.remove('Rosinha Matheus is an Evangelical Christian.')


#erro 'Jador Barbalho is a politician.'
#'Jador Barbalho é um político.'
#sentences.remove('Jador Barbalho is a politician.')

#erro  "Celso Amorim is Arthur Virgílio's neighbour."
#'Celso Amorim é vizinho de Arthur Virgílio.'
#sentences.remove("Celso Amorim is Arthur Virgílio's neighbour.")

#erro 'Souza Aguiar Hospital is situated in Rio de Janeiro.'
#'O Hospital Souza Aguiar está situado no Rio de Janeiro.'
#sentences.remove('Souza Aguiar Hospital is situated in Rio de Janeiro.')

#erro 'Morro da Rocinha is near the Vidigal favela.'
#'O Morro da Rocinha fica perto da favela do Vidigal.'
#sentences.remove('Morro da Rocinha is near the Vidigal favela.')

#erro 'Apathy clouds EU voting.'
#'Apatia nubla votação na UE.'
#sentences.remove('Apathy clouds EU voting.')

#erro 'Dercy Gonçalves died of pneumonia.'
#'Dercy Gonçalves morreu de pneumonia.'
#sentences.remove('Dercy Gonçalves died of pneumonia.')

#erro 'Vítor Miro Bastos de Moura is the employer of Adalberto Xavier Leal.'
#'Vítor Miro Bastos de Moura é o empregador de Adalberto Xavier Leal.'
#sentences.remove('Vítor Miro Bastos de Moura is the employer of Adalberto Xavier Leal.')

In [0]:
import json
with open('rte.json') as reader:
    translation = json.load(reader)

translation['Rosinha Matheus is an Evangelical Christian.'] = 'Rosinha Matheus é um cristã evangélica.'
translation['Jador Barbalho is a politician.'] = 'Jador Barbalho é um político.'
translation["Celso Amorim is Arthur Virgílio's neighbour."] = 'Celso Amorim é vizinho de Arthur Virgílio.'
translation['Souza Aguiar Hospital is situated in Rio de Janeiro.'] = 'O Hospital Souza Aguiar está situado no Rio de Janeiro.'
translation['Morro da Rocinha is near the Vidigal favela.'] = 'O Morro da Rocinha fica perto da favela do Vidigal.'
translation['Apathy clouds EU voting.'] =  'Apatia nubla votação na UE.'
translation['Dercy Gonçalves died of pneumonia.'] = 'Dercy Gonçalves morreu de pneumonia.'
translation['Vítor Miro Bastos de Moura is the employer of Adalberto Xavier Leal.'] = 'Vítor Miro Bastos de Moura é o empregador de Adalberto Xavier Leal.'

with open('rte.json', 'w') as writer:
    json.dump(translation, writer)

Translate

In [0]:
translate2dict(sentences,  'rte.json')

Replace with translation

In [0]:
from os import mkdir
import json

with open('rte.json') as reader:
    dictionary = json.load(reader)

mkdir('translation')
for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, sep = '\t')